In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# Project paths
PROJ_ROOT = Path.cwd().resolve()
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_PROCESSED = PROJ_ROOT / "data" / "processed"

# Ensure write dirs exist
DATA_INTERIM.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)
print("Processed:", DATA_PROCESSED)

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Processed: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\processed


In [2]:
def load_csv_raw(filename: str, **kwargs) -> pd.DataFrame:
    """
    Read a CSV from data/raw by filename only (no absolute paths).
    Example: df = load_csv_raw("series-280625.csv")
    """
    path = DATA_RAW / filename
    # Try utf-8 then latin-1 fallback
    try:
        return pd.read_csv(path, **kwargs)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin-1", **kwargs)

def load_excel_raw(filename: str, sheet_name=0, **kwargs) -> pd.DataFrame:
    """
    Read an Excel sheet from data/raw.
    Example: df = load_excel_raw("ukpopulationestimates183820231.xlsx", sheet_name="MYE")
    """
    path = DATA_RAW / filename
    return pd.read_excel(path, sheet_name=sheet_name, **kwargs)

def save_interim(df: pd.DataFrame, filename: str) -> Path:
    """
    Save a tidy CSV to data/interim.
    Example: save_interim(tidy_df, "gdp_q_tidy.csv")
    """
    out = DATA_INTERIM / filename
    df.to_csv(out, index=False)
    return out

def save_processed(df: pd.DataFrame, filename: str) -> Path:
    """
    Save a processed/combined CSV to data/processed.
    Example: save_processed(master_df, "master_panel.csv")
    """
    out = DATA_PROCESSED / filename
    df.to_csv(out, index=False)
    return out

In [3]:
import re
import warnings
warnings.filterwarnings("ignore", message="Could not infer format")

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^\w_]+", "", regex=True)
        .str.lower()
    )
    return df

def parse_year_quarter_to_period(year, quarter) -> pd.Series:
    y = pd.to_numeric(year, errors="coerce")
    q = pd.to_numeric(quarter, errors="coerce")
    ok = y.notna() & q.notna() & q.between(1,4)
    per = pd.Series(pd.NaT, index=pd.Index(range(len(y))), dtype="datetime64[ns]")
    if ok.any():
        idx = ok[ok].index
        per.loc[idx] = pd.PeriodIndex(
            (y.loc[idx].astype(int).astype(str) + "Q" + q.loc[idx].astype(int).astype(str)).values,
            freq="Q"
        ).to_timestamp(how="end")
    return per

def parse_generic_yq_strings(s: pd.Series) -> pd.Series:
    txt = s.astype(str).str.strip()
    txt = (
        txt.str.replace(r"quarter\s*", "Q", flags=re.I, regex=True)
           .str.replace(r"\s+", "", regex=True)
           .str.replace(r"^Q([1-4])(\d{4})$", r"\2Q\1", regex=True)
    )
    mask = txt.str.match(r"^\d{4}Q[1-4]$", na=False)
    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")
    if mask.any():
        out.loc[mask] = pd.PeriodIndex(txt[mask], freq="Q").to_timestamp(how="end")
    return out

## 1) Annual Defence Expenditure – Combined.xlsx

In [4]:
# --- Imports ---
import re
import numpy as np
import pandas as pd
from pathlib import Path

# (Optional) 'display' safety for non-notebook contexts
try:
    from IPython.display import display
except Exception:
    def display(x): print(x)

# ----------------------------
# Project paths (centralized)
# ----------------------------
PROJ_ROOT = Path.cwd().resolve()           # run notebook from project root
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_PROCESSED = PROJ_ROOT / "data" / "processed"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)
print("Processed:", DATA_PROCESSED)

# ----------------------------
# 1) LOAD & CLEAN (your logic, refined)
# ----------------------------
def load_defence(filepath):
    raw = pd.read_excel(filepath, sheet_name='Annual Defence Spending', header=None)

    # Find header row with FY labels like "2004-05"
    header_row = raw.index[
        raw.apply(lambda r: r.astype(str).str.contains(r'\d{4}-\d{2}').any(), axis=1)
    ][0]
    cols = raw.loc[header_row].tolist()

    # Data starts two rows below the header; restrict to header width
    df = raw.iloc[header_row + 2:, :len(cols)].copy()
    df.columns = ['line'] + cols[1:len(df.columns)]
    df = df.reset_index(drop=True)

    # --- Normalize line labels (strip numbering, tidy variants) ---
    def base_label(s: str):
        s = str(s).replace('\u00a0', ' ')
        s = re.sub(r'^\s*', '', s)
        s = re.sub(r'^\d+(?:\.\d+)?(?:[.)])?\s*', '', s)   # "2", "2.", "2.1", "2)" etc.
        s = re.sub(r'\s*\(\d+\)\s*$', '', s)               # trailing note numbers "(4)"
        s = re.sub(r'\s+', ' ', s).strip().lower()
        s = (s.replace('r & d', 'r&d')
               .replace('r. & d.', 'r&d')
               .replace('r & d.', 'r&d')
               .replace('n. e. c.', 'n.e.c.')
               .replace('n . e . c .', 'n.e.c.'))
        return s

    df['base'] = df['line'].apply(base_label)

    # Locate where the economic block (Current/Capital) starts
    econ_start = df.index[df['base'].isin({'current expenditure', 'capital expenditure'})]
    split_pos = int(econ_start.min()) if len(econ_start) else None

    # Map to final labels; split "total defence" into 1 (functional) and 2 (economic)
    def to_final_label(b, pos):
        if b == 'military defence': return 'Military defence'
        if b == 'civil defence': return 'Civil defence'
        if b == 'foreign military aid': return 'Foreign military aid'
        if b == 'r&d defence': return 'R&D defence'
        if b == 'defence n.e.c.': return 'Defence n.e.c.'
        if b == 'current expenditure': return 'Current Expenditure'
        if b == 'capital expenditure': return 'Capital Expenditure'
        if b == 'total defence':
            if split_pos is not None and pos >= split_pos:
                return 'Total defence 2'   # economic-block total
            else:
                return 'Total defence 1'   # functional-block total
        return None

    df['line_final'] = [to_final_label(b, i) for i, b in enumerate(df['base'])]
    df = df[df['line_final'].notna()]

    # Long -> wide by FY
    long = df.melt(id_vars='line_final', var_name='fy', value_name='value')
    long['fy'] = long['fy'].astype(str).str.extract(r'(\d{4}-\d{2})')[0]
    long = long[long['fy'].notna()]
    long['value'] = pd.to_numeric(long['value'], errors='coerce')

    out = (
        long.pivot_table(index='fy', columns='line_final', values='value', aggfunc='first')
            .reset_index()
            .sort_values('fy')
            .reset_index(drop=True)
    )

    # Robust FY end-year calculation (handles 1999-00 -> 2000 correctly)
    def _fy_endyear(fy):
        m = re.match(r'(\d{4})-(\d{2})', str(fy))
        if not m: return np.nan
        y0 = int(m.group(1))
        y2 = int(m.group(2))
        century = y0 - (y0 % 100)
        endy = century + y2
        if endy < y0:
            endy += 100
        return endy

    out['fy_endyear'] = out['fy'].apply(_fy_endyear)

    # Ensure desired columns exist & order them
    desired = [
        'fy', 'fy_endyear',
        'Military defence', 'Civil defence', 'Foreign military aid', 'R&D defence', 'Defence n.e.c.',
        'Total defence 1', 'Current Expenditure', 'Capital Expenditure', 'Total defence 2'
    ]
    for c in desired:
        if c not in out.columns:
            out[c] = np.nan
    out = out[desired]

    # Strip plan years if present (keep outturns only) – optional
    if len(out) >= 1:
        maybe_last = out.iloc[-1]
        if maybe_last[['Current Expenditure','Capital Expenditure','Total defence 2']].isna().any():
            out = out.iloc[:-1].copy()

    return out


# ----------------------------
# 2) VALIDATION & TIDY ANNUAL
# ----------------------------
def validate_and_make_annual_tidy(df_wide, tolerance=5.0):
    """
    df_wide: output of load_defence()
    tolerance: acceptable absolute gap (in £ mn) for total ≈ current + capital
    """
    df = df_wide.copy()

    # Integrity checks
    df['sum_econ'] = df['Current Expenditure'].fillna(0) + df['Capital Expenditure'].fillna(0)
    df['gap_total_econ'] = (df['Total defence 2'] - df['sum_econ']).abs()

    # Warn on rows that exceed tolerance
    bad = df[df['gap_total_econ'] > tolerance]
    if not bad.empty:
        print("WARNING: Some fiscal years have Total != Current+Capital beyond tolerance.")
        display(bad[['fy','Total defence 2','Current Expenditure','Capital Expenditure','gap_total_econ']])

    # Optional cross-check functional vs economic totals
    if 'Total defence 1' in df.columns:
        df['gap_total12'] = (df['Total defence 2'] - df['Total defence 1']).abs()
        bad2 = df[df['gap_total12'] > tolerance]
        if not bad2.empty:
            print("NOTE: Functional vs economic totals differ beyond tolerance for some years (often rounding/classification).")
            display(bad2[['fy','Total defence 1','Total defence 2','gap_total12']])

    # Build tidy annual (analysis core: total/current/capital, nominal)
    tidy = (
        df[['fy','fy_endyear','Total defence 2','Current Expenditure','Capital Expenditure']]
        .rename(columns={'Total defence 2':'total_nom_mn',
                         'Current Expenditure':'current_nom_mn',
                         'Capital Expenditure':'capital_nom_mn'})
        .melt(id_vars=['fy','fy_endyear'],
              var_name='component', value_name='nominal_mn_gbp')
    )
    tidy['component'] = tidy['component'].str.replace('_nom_mn','', regex=False)
    tidy['component'] = pd.Categorical(tidy['component'], ['total','current','capital'], ordered=True)

    return tidy


# ----------------------------
# 3) FY → QUARTERS MAPPING
# ----------------------------
def fy_to_quarters(fy_str):
    """
    Map FY 'YYYY-YY' to the 4 calendar quarters it spans:
    FY 2004-05 -> ['2004Q2','2004Q3','2004Q4','2005Q1']
    """
    y0 = int(fy_str.split('-')[0])
    qtrs = [f"{y0}Q2", f"{y0}Q3", f"{y0}Q4", f"{y0+1}Q1"]
    return qtrs


# ----------------------------
# 4) QUARTERISATION (indicator-proportional or equal ¼)
# ----------------------------
def quarterise_defence(tidy_annual, gdp_q=None):
    """
    tidy_annual: output of validate_and_make_annual_tidy()
    gdp_q: optional DataFrame with columns ['quarter','gdp_nom'] (YBHA, SA, £ mn)
           If provided, each FY is allocated across its four quarters in proportion to GDP_nom shares.
           If not, we allocate 25% to each quarter.

    Returns: (q_nom_long, q_nom_wide)
    """
    # Build a per-FY × quarter weight frame
    weights_records = []
    all_fy = tidy_annual['fy'].drop_duplicates().tolist()

    if gdp_q is not None:
        gdp_q = gdp_q.copy()
        gdp_q['quarter'] = gdp_q['quarter'].astype(str)

    for fy in all_fy:
        qtrs = fy_to_quarters(fy)
        if gdp_q is not None:
            slice_q = gdp_q[gdp_q['quarter'].isin(qtrs)]
            if slice_q.shape[0] == 4 and slice_q['gdp_nom'].notna().all() and slice_q['gdp_nom'].sum() > 0:
                shares = (slice_q['gdp_nom'] / slice_q['gdp_nom'].sum()).values
            else:
                shares = np.array([0.25,0.25,0.25,0.25])
        else:
            shares = np.array([0.25,0.25,0.25,0.25])

        for q, w in zip(qtrs, shares):
            weights_records.append({'fy': fy, 'quarter': q, 'weight': float(w)})

    weights = pd.DataFrame(weights_records)

    # Allocate each FY × component across its four quarters using weights
    q_nom = (tidy_annual
             .merge(weights, on='fy', how='left')
             .assign(nominal_q_mn=lambda d: d['nominal_mn_gbp'] * d['weight'])
             [['quarter','component','nominal_q_mn']]
            )

    # Wide convenience version too
    q_wide = (q_nom
              .pivot_table(index='quarter', columns='component', values='nominal_q_mn', aggfunc='sum')
              .reset_index()
              .sort_values('quarter')
             )
    # Composition check (optional)
    if set(['current','capital']).issubset(q_wide.columns):
        q_wide['sum_econ'] = q_wide['current'].fillna(0) + q_wide['capital'].fillna(0)
        if 'total' in q_wide.columns:
            gap = (q_wide['total'] - q_wide['sum_econ']).abs()
            if (gap > 1.0).any():
                print("NOTE: At quarterly level, total != current+capital beyond £1m in some quarters (expected small rounding).")

    return q_nom, q_wide


# ----------------------------
# 5) OPTIONAL: DEFLATE QUARTERLY WITH YBGB
# ----------------------------
def deflate_quarterly(q_nom_long, deflator_q, base=100.0):
    """
    q_nom_long: DataFrame with ['quarter','component','nominal_q_mn']
    deflator_q: DataFrame with ['quarter','deflator_index'] (e.g., YBGB, SA)
    base: ensure deflator is already in desired base; pass as-is.

    Returns: long tidy quarterly with real series and logs.
    """
    df = (q_nom_long.merge(deflator_q, on='quarter', how='left')
                    .rename(columns={'deflator_index':'defl'}))
    if df['defl'].isna().any():
        missing = df[df['defl'].isna()]['quarter'].unique()[:10]
        print("WARNING: Missing deflator for some quarters (showing up to 10):", missing)

    df['real_q_mn'] = df['nominal_q_mn'] / (df['defl'] / base)
    df['ln_real'] = np.log(df['real_q_mn'].replace({0: np.nan}))
    return df


# ----------------------------
# 6) RUN THE PIPELINE (reads from data/raw, writes to data/interim)
# ----------------------------
# Locate the defence workbook in /data/raw (handles different dash characters)
defence_file = None
for p in list(DATA_RAW.glob("*.xlsx")) + list(DATA_RAW.glob("*.xls")):
    nm = p.name.lower().replace("—", "-").replace("–", "-")
    if ("annual defence expenditure" in nm and "combined" in nm) or \
       ("annual_defence_expenditure" in nm and "combined" in nm):
        defence_file = p
        break

# Fallback to exact name if you prefer
if defence_file is None:
    fallback = DATA_RAW / "Annual Defence Expenditure – Combined.xlsx"
    if fallback.exists():
        defence_file = fallback

if defence_file is None:
    raise FileNotFoundError(
        "Could not find 'Annual Defence Expenditure – Combined.xlsx' in data/raw. "
        "Please place the file there (name can include hyphen/en dash)."
    )

print(f"Using defence workbook: {defence_file}")

# Load & clean
df_wide = load_defence(defence_file)

# Validate & produce tidy annual
annual_tidy = validate_and_make_annual_tidy(df_wide, tolerance=5.0)

# Save annual tidy to /data/interim
annual_out = DATA_INTERIM / 'defence_fy_tidy.csv'
annual_tidy.to_csv(annual_out, index=False)
print(f"Saved tidy annual to: {annual_out}")

# ---------- Quarterise ----------
# OPTION A (recommended): pass quarterly nominal GDP (YBHA) for proportional allocation
# If/when available in data/raw, load as below:
# ybha_file = next((p for p in DATA_RAW.glob("series-*.csv") if "ybha" in p.name.lower()), None)
# if ybha_file:
#     ybha_raw = pd.read_csv(ybha_file)
#     gdp_q = (ybha_raw.rename(columns={'Period':'quarter','Value':'gdp_nom'})[['quarter','gdp_nom']])
#     gdp_q['gdp_nom'] = pd.to_numeric(gdp_q['gdp_nom'], errors='coerce')
# else:
#     gdp_q = None

gdp_q = None  # leave None for now; equal ¼ allocation

q_nom_long, q_nom_wide = quarterise_defence(annual_tidy, gdp_q=gdp_q)

# Save quarterly nominal to /data/interim
q_nom_long_out = DATA_INTERIM / 'defence_q_nominal_long.csv'
q_nom_wide_out = DATA_INTERIM / 'defence_q_nominal_wide.csv'
q_nom_long.to_csv(q_nom_long_out, index=False)
q_nom_wide.to_csv(q_nom_wide_out, index=False)
print(f"Saved quarterly nominal (long) to: {q_nom_long_out}")
print(f"Saved quarterly nominal (wide) to: {q_nom_wide_out}")

# ---------- Deflate (optional; do this before modelling) ----------
# If/when YBGB deflator CSV is in data/raw, load as below and then deflate:
# ybgb_file = next((p for p in DATA_RAW.glob("series-*.csv") if "ybgb" in p.name.lower()), None)
# if ybgb_file:
#     ybgb_raw = pd.read_csv(ybgb_file)
#     deflator_q = (ybgb_raw.rename(columns={'Period':'quarter','Value':'deflator_index'})[['quarter','deflator_index']])
#     deflator_q['deflator_index'] = pd.to_numeric(deflator_q['deflator_index'], errors='coerce')
#     q_real_long = deflate_quarterly(q_nom_long, deflator_q, base=100.0)
#     q_real_out = DATA_INTERIM / 'defence_q_real_long.csv'
#     q_real_long.to_csv(q_real_out, index=False)
#     print(f"Saved quarterly REAL (long) to: {q_real_out}")

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Processed: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\processed
Using defence workbook: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw\Annual Defence Expenditure - Combined.xlsx
Saved tidy annual to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\defence_fy_tidy.csv
Saved quarterly nominal (long) to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\defence_q_nominal_long.csv
Saved quarterly nominal (wide) to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\defence_q_nominal_wide.csv


C:\Users\Aniruddha\AppData\Local\Temp\ipykernel_25204\111195262.py:231: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  q_wide = (q_nom


## 2) series-010725.csv (ONS code YBHA)

In [5]:
from pathlib import Path
import csv, re, datetime as dt
import pandas as pd
import numpy as np
from IPython.display import display

# ----------------------------
# Project paths
# ----------------------------
PROJ_ROOT = Path.cwd().resolve()
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)

# ----------------------------
# 0) CONFIG
# ----------------------------
# Find the YBHA file in data/raw (accepts e.g. 'series-010725.csv' or variants)
ybha_file = DATA_RAW / "series-010725.csv"
if not ybha_file.exists():
    # fallback to any close match (e.g., duplicates like 'series-010725 (1).csv')
    candidates = sorted(DATA_RAW.glob("series-010725*.csv"))
    if not candidates:
        raise FileNotFoundError("Could not find 'series-010725.csv' in data/raw.")
    ybha_file = candidates[0]

print(f"Using YBHA source: {ybha_file}")

# Analysis window (quarters, inclusive)
ANALYSIS_START_Q = '2004Q1'   # keep Q1 so FY 2004-05 coverage (Q2–Q4 + next Q1) is possible
ANALYSIS_END_Q   = '2024Q4'   # safe upper bound; you can trim later when merging

# ----------------------------
# 1) LOAD & PARSE (your logic, wrapped and extended)
# ----------------------------
def load_ons_single_series(path: Path) -> pd.DataFrame:
    """
    Load ONS 'single series' CSV (e.g., YBHA).
    Returns tidy df with: period(str), value(float), year(int), quarter(Int64)
    """
    text = path.read_text(encoding='utf-8', errors='ignore').splitlines()
    rows = [next(csv.reader([ln])) for ln in text if ln.strip()]
    data = [(r[0].strip('"'), r[1]) for r in rows
            if len(r) >= 2 and re.match(r'^\d{4}(?:\s+Q[1-4])?$', r[0].strip('"'))]
    df = pd.DataFrame(data, columns=['period', 'value'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['year'] = df['period'].str.extract(r'(\d{4})').astype(int)
    df['quarter'] = df['period'].str.extract(r'Q([1-4])').astype('Int64')
    return df

ybha = load_ons_single_series(ybha_file)

# Keep QUARTERLY rows only; drop annual (not needed for weights)
ybha_q = (ybha[ybha['quarter'].notna()]
          .rename(columns={'value':'gdp_nom_mn'})
          .copy()
         )

# Canonical quarter key and period end date
ybha_q['quarter_key'] = ybha_q['year'].astype(int).astype(str) + 'Q' + ybha_q['quarter'].astype(int).astype(str)

def quarter_end_date(y, q):
    month = {1:3, 2:6, 3:9, 4:12}[int(q)]
    return dt.date(int(y), month, 1) + pd.offsets.MonthEnd(0)

ybha_q['quarter_end'] = [quarter_end_date(y, q) for y, q in zip(ybha_q['year'], ybha_q['quarter'])]

# Trim to analysis window
def qrank(qstr):  # e.g., 2004Q2 -> 2004*10 + 2
    y, q = qstr.split('Q')
    return int(y)*10 + int(q)

mask = (ybha_q['quarter_key'].apply(qrank) >= qrank(ANALYSIS_START_Q)) & \
       (ybha_q['quarter_key'].apply(qrank) <= qrank(ANALYSIS_END_Q))
ybha_q = ybha_q.loc[mask].reset_index(drop=True)

# ----------------------------
# 2) MAP TO FISCAL YEARS & BUILD WITHIN-FY WEIGHTS
# ----------------------------
def to_fy(qkey: str) -> str:
    y, q = qkey.split('Q')
    y = int(y); q = int(q)
    # FY runs Apr–Mar; Q2–Q4 belong to FY starting in that year; Q1 belongs to FY starting previous year
    if q == 1:
        fy_start = y - 1
    else:
        fy_start = y
    fy_end_two = str((fy_start + 1) % 100).zfill(2)
    return f"{fy_start}-{fy_end_two}"

ybha_q['fy'] = ybha_q['quarter_key'].apply(to_fy)

# FY totals and weights
fy_totals = (ybha_q.groupby('fy', as_index=False)['gdp_nom_mn']
             .sum().rename(columns={'gdp_nom_mn':'fy_gdp_nom_sum'}))

ybha_w = (ybha_q.merge(fy_totals, on='fy', how='left')
                .assign(ybha_weight=lambda d: d['gdp_nom_mn'] / d['fy_gdp_nom_sum'])
                [['fy','quarter_key','gdp_nom_mn','fy_gdp_nom_sum','ybha_weight']]
          )

# Sanity: each FY should have exactly 4 quarters, weights sum to ~1
check = (ybha_w.groupby('fy')
         .agg(n_quarters=('quarter_key','nunique'),
              weight_sum=('ybha_weight','sum')))
bad_fy = check[(check['n_quarters'] != 4) | (check['weight_sum'].sub(1).abs() > 1e-6)]
if not bad_fy.empty:
    print("WARNING: FY coverage/weight-sum issues detected:")
    display(bad_fy.head(10))

# ----------------------------
# 3) SAVE CLEAN OUTPUTS (to data/interim)
# ----------------------------
# 3a) Quarterly GDP (clean)
ybha_q_out = (ybha_q[['quarter_key','quarter_end','gdp_nom_mn']]
              .assign(cdid='YBHA')
              .rename(columns={'quarter_key':'quarter'}))

f_q = DATA_INTERIM / 'ybha_q.csv'
ybha_q_out.to_csv(f_q, index=False)
print(f"Saved quarterly nominal GDP to: {f_q}")

# 3b) FY weights (for quarterising defence)
ybha_w_out = ybha_w.rename(columns={'quarter_key':'quarter'})
f_w = DATA_INTERIM / 'ybha_fy_weights.csv'
ybha_w_out.to_csv(f_w, index=False)
print(f"Saved FY weights to: {f_w}")

# ----------------------------
# 4) OPTIONAL: QUICK DISPLAY / QA
# ----------------------------
print("\nSample (quarterly GDP):")
display(ybha_q_out.head(8))
print("Sample (FY weights):")
display(ybha_w_out.head(8))

# Optional deeper QA: show one FY’s weights sum to 1
if not ybha_w_out.empty:
    ex_fy = ybha_w_out['fy'].iloc[0]
    print(f"Weight sum check for FY {ex_fy}:",
          ybha_w_out.query("fy == @ex_fy")['ybha_weight'].sum())

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Using YBHA source: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw\series-010725.csv


,n_quarters,weight_sum
fy,,
2003-04,1,1.0
2024-25,3,1.0


Saved quarterly nominal GDP to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\ybha_q.csv
Saved FY weights to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\ybha_fy_weights.csv

Sample (quarterly GDP):


,quarter,quarter_end,gdp_nom_mn,cdid
0,2004Q1,2004-03-31,322778,YBHA
1,2004Q2,2004-06-30,329732,YBHA
2,2004Q3,2004-09-30,331592,YBHA
3,2004Q4,2004-12-31,338535,YBHA
4,2005Q1,2005-03-31,341254,YBHA
5,2005Q2,2005-06-30,348491,YBHA
6,2005Q3,2005-09-30,352001,YBHA
7,2005Q4,2005-12-31,357003,YBHA


Sample (FY weights):


,fy,quarter,gdp_nom_mn,fy_gdp_nom_sum,ybha_weight
0,2003-04,2004Q1,322778,322778,1.000000
1,2004-05,2004Q2,329732,1341113,0.245864
2,2004-05,2004Q3,331592,1341113,0.247251
3,2004-05,2004Q4,338535,1341113,0.252428
4,2004-05,2005Q1,341254,1341113,0.254456
5,2005-06,2005Q2,348491,1418708,0.245640
6,2005-06,2005Q3,352001,1418708,0.248114
7,2005-06,2005Q4,357003,1418708,0.251640


Weight sum check for FY 2003-04: 1.0


## 3) series-280625.csv (ONS code ABMI)

In [6]:
from pathlib import Path
import csv, re, datetime as dt
import pandas as pd
import numpy as np
from IPython.display import display

# ----------------------------
# Project paths
# ----------------------------
PROJ_ROOT = Path.cwd().resolve()
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)

# ----------------------------
# 0) CONFIG (now sourced from data/raw)
# ----------------------------
# Prefer exact name; fall back to close matches if needed
abmi_file = DATA_RAW / "series-280625.csv"
if not abmi_file.exists():
    candidates = sorted(DATA_RAW.glob("series-280625*.csv"))
    if not candidates:
        raise FileNotFoundError("Could not find 'series-280625.csv' in data/raw.")
    abmi_file = candidates[0]
print(f"Using ABMI source: {abmi_file}")

ANALYSIS_START_Q = '1997Q1'   # generous buffer for lags/diagnostics
ANALYSIS_END_Q   = '2025Q4'   # safe upper bound; you’ll trim to overlap later

# ----------------------------
# 1) LOAD & PARSE (your base, extended)
# ----------------------------
def load_ons_single_series(p: Path) -> pd.DataFrame:
    """
    Load an ONS 'single series' CSV (e.g., ABMI).
    Returns tidy df with: period, value, year, quarter(Int64)
    """
    text = p.read_text(encoding='utf-8', errors='ignore').splitlines()
    rows = [next(csv.reader([ln])) for ln in text if ln.strip()]
    # Keep "YYYY" or "YYYY Qn"
    data = [(r[0].strip('"'), r[1]) for r in rows
            if len(r) >= 2 and re.match(r'^\d{4}(?:\s+Q[1-4])?$', r[0].strip('"'))]
    df = pd.DataFrame(data, columns=['period', 'value'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['year'] = df['period'].str.extract(r'(\d{4})').astype(int)
    df['quarter'] = df['period'].str.extract(r'Q([1-4])').astype('Int64')
    return df

abmi_raw = load_ons_single_series(abmi_file)

# Keep QUARTERLY rows only (drop annual)
abmi_q = (abmi_raw[abmi_raw['quarter'].notna()]
          .rename(columns={'value':'gdp_real_mn'})
          .copy())

# Canonical quarter key
abmi_q['quarter'] = (
    abmi_q['year'].astype(int).astype(str) + 'Q' + abmi_q['quarter'].astype(int).astype(str)
)

# End-of-quarter date (handy for plotting/joins)
def quarter_end_date(qkey: str):
    y, q = qkey.split('Q')
    y = int(y); q = int(q)
    month = {1:3, 2:6, 3:9, 4:12}[q]
    return pd.Timestamp(year=y, month=month, day=1) + pd.offsets.MonthEnd(0)

abmi_q['quarter_end'] = abmi_q['quarter'].apply(quarter_end_date)

# ----------------------------
# 2) TRIM WINDOW & SORT
# ----------------------------
def qrank(qstr):  # e.g. 2004Q2 -> 20042
    y, q = qstr.split('Q')
    return int(y)*10 + int(q)

mask = (abmi_q['quarter'].apply(qrank) >= qrank(ANALYSIS_START_Q)) & \
       (abmi_q['quarter'].apply(qrank) <= qrank(ANALYSIS_END_Q))
abmi_q = abmi_q.loc[mask].sort_values('quarter').reset_index(drop=True)

# ----------------------------
# 3) DERIVED FIELDS (logs, growths, technical recession)
# ----------------------------
abmi_q['ln_gdp_real'] = np.log(abmi_q['gdp_real_mn'].replace({0: np.nan}))

# log-diff growths (×100 for %)
abmi_q['g_qoq_pct'] = 100 * (abmi_q['ln_gdp_real'] - abmi_q['ln_gdp_real'].shift(1))
abmi_q['g_yoy_pct'] = 100 * (abmi_q['ln_gdp_real'] - abmi_q['ln_gdp_real'].shift(4))

# Technical recession dummy: two consecutive negative q/q
neg = (abmi_q['g_qoq_pct'] < 0).astype(int)
abmi_q['rec_tech'] = ((neg.shift(0) == 1) & (neg.shift(1) == 1)).astype(int)

# Optional period flags (convenient labels for splits)
abmi_q['post_gfc']     = (abmi_q['quarter'].apply(qrank) >= qrank('2008Q3'))
abmi_q['post_brexit']  = (abmi_q['quarter'].apply(qrank) >= qrank('2016Q3'))
abmi_q['covid_window'] = (abmi_q['quarter'].apply(qrank) >= qrank('2020Q2')) & \
                         (abmi_q['quarter'].apply(qrank) <= qrank('2022Q1'))

# ----------------------------
# 4) QA CHECKS
# ----------------------------
# 4a) Duplicates / gaps
dups = abmi_q.duplicated('quarter').sum()
if dups:
    print(f"WARNING: {dups} duplicate quarter keys.")

# 4b) Missing values
if abmi_q['gdp_real_mn'].isna().any():
    print("WARNING: Missing ABMI values after parse/trim.")

# 4c) Quick sanity preview
print("ABMI sample after cleaning:")
display(abmi_q.head(8))
display(abmi_q.tail(8))

# ----------------------------
# 5) SAVE CLEAN OUTPUT (to data/interim)
# ----------------------------
out_cols = [
    'quarter','quarter_end','gdp_real_mn','ln_gdp_real',
    'g_qoq_pct','g_yoy_pct','rec_tech',
    'post_gfc','post_brexit','covid_window'
]
abmi_out = abmi_q[out_cols].copy()

f_out = DATA_INTERIM / 'abmi_q.csv'
abmi_out.to_csv(f_out, index=False)
print(f"Saved cleaned ABMI to: {f_out}")

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Using ABMI source: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw\series-280625.csv
ABMI sample after cleaning:


,period,gdp_real_mn,year,quarter,quarter_end,ln_gdp_real,g_qoq_pct,g_yoy_pct,rec_tech,post_gfc,post_brexit,covid_window
0,1997 Q1,397303,1997,1997Q1,1997-03-31,12.892454,NaN,NaN,0,False,False,False
1,1997 Q2,401790,1997,1997Q2,1997-06-30,12.903685,1.123035,NaN,0,False,False,False
2,1997 Q3,405233,1997,1997Q3,1997-09-30,12.912217,0.853265,NaN,0,False,False,False
3,1997 Q4,411352,1997,1997Q4,1997-12-31,12.927205,1.498709,NaN,0,False,False,False
4,1998 Q1,414377,1998,1998Q1,1998-03-31,12.934531,0.732689,4.207697,0,False,False,False
5,1998 Q2,416909,1998,1998Q2,1998-06-30,12.940623,0.609179,3.693841,0,False,False,False
6,1998 Q3,418294,1998,1998Q3,1998-09-30,12.943940,0.331656,3.172232,0,False,False,False
7,1998 Q4,421074,1998,1998Q4,1998-12-31,12.950564,0.662406,2.335929,0,False,False,False


,period,gdp_real_mn,year,quarter,quarter_end,ln_gdp_real,g_qoq_pct,g_yoy_pct,rec_tech,post_gfc,post_brexit,covid_window
105,2023 Q2,634694,2023,2023Q2,2023-06-30,13.360898,0.041919,0.542834,0,True,True,False
106,2023 Q3,634327,2023,2023Q3,2023-09-30,13.360320,-0.057840,0.371159,0,True,True,False
107,2023 Q4,633011,2023,2023Q4,2023-12-31,13.358243,-0.207679,-0.163055,1,True,True,False
108,2024 Q1,638746,2024,2024Q1,2024-03-31,13.367262,0.901908,0.678307,0,True,True,False
109,2024 Q2,641670,2024,2024Q2,2024-06-30,13.371829,0.456727,1.093116,0,True,True,False
110,2024 Q3,641675,2024,2024Q3,2024-09-30,13.371837,0.000779,1.151735,0,True,True,False
111,2024 Q4,642287,2024,2024Q4,2024-12-31,13.372791,0.095330,1.454744,0,True,True,False
112,2025 Q1,646833,2025,2025Q1,2025-03-31,13.379843,0.705290,1.258127,0,True,True,False


Saved cleaned ABMI to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\abmi_q.csv


## 4) series-030725.csv (ONS code YBGB – GDP implied deflator index)

In [7]:
from pathlib import Path
import csv, re
import pandas as pd
import numpy as np
from IPython.display import display

# --------------------------------
# Project paths
# --------------------------------
PROJ_ROOT = Path.cwd().resolve()
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)

# --------------------------------
# 0) CONFIG — source files from data/raw
# --------------------------------
# Prefer exact name; fall back to close matches if needed
ybgb_file = DATA_RAW / "series-030725.csv"
if not ybgb_file.exists():
    candidates = sorted(DATA_RAW.glob("series-030725*.csv"))
    if not candidates:
        raise FileNotFoundError("Could not find 'series-030725.csv' in data/raw.")
    ybgb_file = candidates[0]
print(f"Using YBGB source: {ybgb_file}")

# Optional: if you extract Annex F FY deflators to a CSV with columns: fy, annexF_deflator
# (fy formatted like "2004-05"), put that CSV in data/raw and set its filename below.
annex_f_csv = None  # e.g., "annexF_deflators_fy.csv"

ANALYSIS_START_Q = '1997Q1'
ANALYSIS_END_Q   = '2025Q4'

# --------------------------------
# 1) LOAD & PARSE
# --------------------------------
def load_ons_single_series(p: Path) -> pd.DataFrame:
    """
    Load an ONS 'single series' CSV (e.g., YBGB).
    Returns tidy df with: period, value, year, quarter(Int64)
    """
    text = p.read_text(encoding='utf-8', errors='ignore').splitlines()
    rows = [next(csv.reader([ln])) for ln in text if ln.strip()]
    data = [(r[0].strip('"'), r[1]) for r in rows
            if len(r) >= 2 and re.match(r'^\d{4}(?:\s+Q[1-4])?$', r[0].strip('"'))]
    df = pd.DataFrame(data, columns=['period', 'value'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')  # index value (e.g., CY=100)
    df['year'] = df['period'].str.extract(r'(\d{4})').astype(int)
    df['quarter'] = df['period'].str.extract(r'Q([1-4])').astype('Int64')
    return df

ybgb_raw = load_ons_single_series(ybgb_file)

# Keep QUARTERLY only
ybgb_q = (ybgb_raw[ybgb_raw['quarter'].notna()]
          .rename(columns={'value':'deflator_index'})
          .copy())

# Canonical quarter key and end-of-quarter date
ybgb_q['quarter'] = ybgb_q['year'].astype(str) + 'Q' + ybgb_q['quarter'].astype(int).astype(str)

def quarter_end_date(qkey: str):
    y, q = qkey.split('Q')
    y = int(y); q = int(q)
    month = {1:3, 2:6, 3:9, 4:12}[q]
    return pd.Timestamp(year=y, month=month, day=1) + pd.offsets.MonthEnd(0)

ybgb_q['quarter_end'] = ybgb_q['quarter'].apply(quarter_end_date)

# Trim window
def qrank(qstr):  # e.g. 2004Q2 -> 20042
    y, q = qstr.split('Q')
    return int(y)*10 + int(q)

mask = (ybgb_q['quarter'].apply(qrank) >= qrank(ANALYSIS_START_Q)) & \
       (ybgb_q['quarter'].apply(qrank) <= qrank(ANALYSIS_END_Q))
ybgb_q = ybgb_q.loc[mask].sort_values('quarter').reset_index(drop=True)

# --------------------------------
# 2) DIAGNOSTICS & FY MAPPING
# --------------------------------
# Inflation diagnostics (log-diff %)
ybgb_q['ln_defl']    = np.log(ybgb_q['deflator_index'].replace({0: np.nan}))
ybgb_q['pi_qoq_pct'] = 100 * (ybgb_q['ln_defl'] - ybgb_q['ln_defl'].shift(1))
ybgb_q['pi_yoy_pct'] = 100 * (ybgb_q['ln_defl'] - ybgb_q['ln_defl'].shift(4))

# Fiscal year mapping (Apr–Mar): FY YYYY-YY = Q2..Q4 of YYYY plus Q1 of YYYY+1
def to_fy(qkey: str) -> str:
    y, q = qkey.split('Q'); y = int(y); q = int(q)
    fy_start = y - 1 if q == 1 else y
    fy_end_two = str((fy_start + 1) % 100).zfill(2)
    return f"{fy_start}-{fy_end_two}"

ybgb_q['fy'] = ybgb_q['quarter'].apply(to_fy)

# FY averages of YBGB (for checks/anchoring)
fy_avg = (ybgb_q.groupby('fy', as_index=False)['deflator_index']
          .mean().rename(columns={'deflator_index':'mean_ybgb_quarterly'}))

# --------------------------------
# 3) OPTIONAL: PESA-ANCHORED VARIANT (if Annex F FY deflators provided)
# --------------------------------
anchored = None
fy_check = None

if annex_f_csv is not None:
    annex_path = DATA_RAW / annex_f_csv
    if annex_path.exists():
        annex = pd.read_csv(annex_path)
        # Expect columns: fy, annexF_deflator
        assert {'fy','annexF_deflator'}.issubset(annex.columns), \
            "Annex-F CSV must have columns: fy, annexF_deflator"
        # Merge FY average YBGB with Annex F to get scaling factors by FY
        fy_check = (fy_avg.merge(annex, on='fy', how='inner')
                          .assign(scale_fy=lambda d: d['annexF_deflator'] / d['mean_ybgb_quarterly'],
                                  ratio=lambda d: d['mean_ybgb_quarterly'] / d['annexF_deflator']))
        # Apply FY-specific scale to each quarter in that FY
        anchored = (ybgb_q.merge(fy_check[['fy','scale_fy']], on='fy', how='left')
                         .assign(deflator_index_fyanchored=lambda d: d['deflator_index'] * d['scale_fy']))
    else:
        print(f"FY-anchored variant skipped (Annex-F CSV not found: {annex_path})")
else:
    print("FY-anchored variant skipped (set 'annex_f_csv' to a CSV with FY Annex-F deflators in data/raw).")

# --------------------------------
# 4) QA
# --------------------------------
# Contiguity per FY (should be 4 quarters)
fy_counts = ybgb_q.groupby('fy')['quarter'].nunique()
bad_fy = fy_counts[fy_counts != 4]
if not bad_fy.empty:
    print("WARNING: Some FYs do not have exactly 4 quarters:\n", bad_fy)

# Quick preview
print("YBGB sample after cleaning:")
display(ybgb_q.head(8))

# --------------------------------
# 5) SAVE OUTPUTS (to data/interim)
# --------------------------------
# 5a) As-published quarterly deflator
ybgb_out = ybgb_q[['quarter','quarter_end','deflator_index','pi_qoq_pct','pi_yoy_pct','fy']]
f_q = DATA_INTERIM / 'ybgb_q.csv'
ybgb_out.to_csv(f_q, index=False)
print(f"Saved as-published quarterly deflator to: {f_q}")

# 5b) FY-anchored quarterly deflator (if available)
if anchored is not None:
    ybgb_anch = anchored[['quarter','fy','deflator_index_fyanchored']]
    f_a = DATA_INTERIM / 'ybgb_q_fyanchored.csv'
    ybgb_anch.to_csv(f_a, index=False)
    print(f"Saved FY-anchored quarterly deflator to: {f_a}")
    # FY check table
    if fy_check is not None:
        f_check = DATA_INTERIM / 'ybgb_fy_check.csv'
        fy_check.to_csv(f_check, index=False)
        print(f"Saved FY anchoring check table to: {f_check}")

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Using YBGB source: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw\series-030725.csv
FY-anchored variant skipped (set 'annex_f_csv' to a CSV with FY Annex-F deflators in data/raw).
 fy
1996-97    1
Name: quarter, dtype: int64
YBGB sample after cleaning:


,period,deflator_index,year,quarter,quarter_end,ln_defl,pi_qoq_pct,pi_yoy_pct,fy
0,1997 Q1,58.6,1997,1997Q1,1997-03-31,4.070735,NaN,NaN,1996-97
1,1997 Q2,58.7,1997,1997Q2,1997-06-30,4.072440,0.170503,NaN,1997-98
2,1997 Q3,60.1,1997,1997Q3,1997-09-30,4.096010,2.357011,NaN,1997-98
3,1997 Q4,58.7,1997,1997Q4,1997-12-31,4.072440,-2.357011,NaN,1997-98
4,1998 Q1,59.4,1998,1998Q1,1998-03-31,4.084294,1.185450,1.355953,1997-98
5,1998 Q2,59.5,1998,1998Q2,1998-06-30,4.085976,0.168209,1.353659,1998-99
6,1998 Q3,59.9,1998,1998Q3,1998-09-30,4.092677,0.670019,-0.333334,1998-99
7,1998 Q4,60.3,1998,1998Q4,1998-12-31,4.099332,0.665560,2.689238,1998-99


Saved as-published quarterly deflator to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\ybgb_q.csv


## 5) series-030725 (1).csv (ONS code L522 – CPIH index, 2015=100, monthly)

In [8]:
from pathlib import Path
import csv, re, calendar, datetime as dt
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display

# --------------------------------
# Project paths
# --------------------------------
PROJ_ROOT = Path.cwd().resolve()
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)

# --------------------------------
# 0) CONFIG — source from data/raw
# --------------------------------
# Prefer exact name; fall back to close matches if needed
cpih_file = DATA_RAW / "series-030725 (1).csv"
if not cpih_file.exists():
    candidates = sorted(DATA_RAW.glob("series-030725*1*.csv")) or \
                 sorted(DATA_RAW.glob("series-030725*.csv"))
    if not candidates:
        raise FileNotFoundError("Could not find 'series-030725 (1).csv' in data/raw.")
    cpih_file = candidates[0]
print(f"Using CPIH (L522) source: {cpih_file}")

ANALYSIS_START_Q = '1997Q1'   # generous buffer
ANALYSIS_END_Q   = '2025Q4'   # safe cap; final merge will align windows

# --------------------------------
# 1) HELPERS (your parser, extended)
# --------------------------------
def parse_period_monthly(s: str):
    """
    Parse a monthly ONS period string into (year, month).
    Accepts '1988 JAN', '1988 January', '1988-01', '1988 01', '1988/01', '1988-Jan'.
    """
    s = s.strip().strip('"').replace('\u00a0', ' ')
    for fmt in ("%Y %b", "%Y %B", "%Y-%m", "%Y %m", "%Y/%m", "%Y-%b", "%Y/%b"):
        try:
            dtm = datetime.strptime(s, fmt)
            return dtm.year, dtm.month
        except Exception:
            pass
    m = re.match(r'^(\d{4})\s+([A-Za-z]{3,})$', s)
    if m:
        year = int(m.group(1))
        mon_name = m.group(2).capitalize()
        month_map = {name: i for i, name in enumerate(calendar.month_name) if name}
        month_map.update({name: i for i, name in enumerate(calendar.month_abbr) if name})
        if mon_name in month_map:
            return year, month_map[mon_name]
    m = re.match(r'^(\d{4})[-/](\d{1,2})$', s) or re.match(r'^(\d{4})\s+(\d{1,2})$', s)
    if m:
        return int(m.group(1)), int(m.group(2))
    return None, None

def load_ons_single_series_monthly(p: Path) -> pd.DataFrame:
    """
    Load ONS 'single series' CSV (monthly). Return tidy monthly df.
    Columns: period, cpih_idx, year, month, month_end
    """
    lines = p.read_text(encoding='utf-8', errors='ignore').splitlines()
    rows = [next(csv.reader([ln])) for ln in lines if ln.strip()]

    # Keep rows that likely hold data (start with a year)
    data = [(r[0].strip(), r[1]) for r in rows if len(r) >= 2 and re.match(r'^\s*"?\d{4}', r[0])]
    periods, values, years, months = [], [], [], []
    for pstr, val in data:
        y, m = parse_period_monthly(pstr)
        if y is None or m is None:
            continue
        periods.append(pstr.strip().strip('"'))
        values.append(val)
        years.append(y)
        months.append(m)

    df = pd.DataFrame({'period': periods, 'cpih_idx': values, 'year': years, 'month': months})
    df['cpih_idx'] = pd.to_numeric(df['cpih_idx'], errors='coerce')
    df['month_end'] = pd.to_datetime(df[['year', 'month']].assign(day=1)) + pd.offsets.MonthEnd(0)
    df = df.sort_values(['year', 'month']).reset_index(drop=True)
    return df

def qkey_from_date(ts: pd.Timestamp) -> str:
    q = (ts.month - 1)//3 + 1
    return f"{ts.year}Q{q}"

def qrank(qstr: str) -> int:
    y, q = qstr.split('Q')
    return int(y)*10 + int(q)

def quarter_end_date_from_key(qkey: str) -> pd.Timestamp:
    y, q = qkey.split('Q'); y = int(y); q = int(q)
    month = {1:3, 2:6, 3:9, 4:12}[q]
    return pd.Timestamp(year=y, month=month, day=1) + pd.offsets.MonthEnd(0)

# --------------------------------
# 2) LOAD MONTHLY + DIAGNOSTICS
# --------------------------------
cpih_m = load_ons_single_series_monthly(cpih_file)

# Monthly YoY (prefer for charts; MoM optional)
cpih_m['ln_idx'] = np.log(cpih_m['cpih_idx'].replace({0: np.nan}))
cpih_m['inf_yoy_pct_m'] = 100 * (cpih_m['ln_idx'] - cpih_m['ln_idx'].shift(12))
cpih_m['inf_mom_pct_m'] = 100 * (cpih_m['ln_idx'] - cpih_m['ln_idx'].shift(1))

# Save monthly file (QA/plots) -> data/interim
cpih_monthly_out = cpih_m[['period','month_end','cpih_idx','inf_yoy_pct_m','inf_mom_pct_m']].copy()
cpih_monthly_out['month'] = cpih_monthly_out['month_end'].dt.strftime('%Y-%m')
cpih_monthly_out = cpih_monthly_out[['month','month_end','cpih_idx','inf_yoy_pct_m','inf_mom_pct_m']]
f_m = DATA_INTERIM / 'cpih_monthly.csv'
cpih_monthly_out.to_csv(f_m, index=False)
print(f"Saved CPIH monthly to: {f_m}")

# --------------------------------
# 3) AGGREGATE TO QUARTERS (analysis dataset)
# --------------------------------
cpih_m['quarter'] = cpih_m['month_end'].apply(qkey_from_date)
# Average index over 3 months per quarter (robust standard)
q_agg = (cpih_m.groupby('quarter', as_index=False)
         .agg(cpih_idx_q=('cpih_idx','mean'),
              n_months=('cpih_idx','size')))

# Partial-quarter flag
q_agg['is_partial_q'] = (q_agg['n_months'] < 3).astype(int)

# Add quarter_end and sort
q_agg['quarter_end'] = q_agg['quarter'].apply(quarter_end_date_from_key)
q_agg = q_agg.sort_values('quarter').reset_index(drop=True)

# Trim window
mask = (q_agg['quarter'].apply(qrank) >= qrank(ANALYSIS_START_Q)) & \
       (q_agg['quarter'].apply(qrank) <= qrank(ANALYSIS_END_Q))
q_agg = q_agg.loc[mask].reset_index(drop=True)

# Quarterly inflation rates (YoY preferred for modelling; QoQ optional)
q_agg['ln_idx_q'] = np.log(q_agg['cpih_idx_q'].replace({0: np.nan}))
q_agg['inf_qoq_pct_q'] = 100 * (q_agg['ln_idx_q'] - q_agg['ln_idx_q'].shift(1))
q_agg['inf_yoy_pct_q'] = 100 * (q_agg['ln_idx_q'] - q_agg['ln_idx_q'].shift(4))

# Final quarterly output
cpih_quarterly_out = q_agg[['quarter','quarter_end','cpih_idx_q','inf_qoq_pct_q','inf_yoy_pct_q','is_partial_q']].copy()

# Warn if last quarter is partial; you’ll typically drop partials in modelling
if not cpih_quarterly_out.empty and cpih_quarterly_out['is_partial_q'].iloc[-1] == 1:
    print("Note: Latest quarter is partial (fewer than 3 months). Consider excluding from estimation.")

# Save quarterly file -> data/interim
f_q = DATA_INTERIM / 'cpih_quarterly.csv'
cpih_quarterly_out.to_csv(f_q, index=False)
print(f"Saved CPIH quarterly to: {f_q}")

# --------------------------------
# 4) QUICK PREVIEWS
# --------------------------------
print("\nMonthly sample:")
display(cpih_monthly_out.head(12))
print("Quarterly sample:")
display(cpih_quarterly_out.head(8))

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Using CPIH (L522) source: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw\series-030725 (1).csv
Saved CPIH monthly to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\cpih_monthly.csv
Note: Latest quarter is partial (fewer than 3 months). Consider excluding from estimation.
Saved CPIH quarterly to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\cpih_quarterly.csv

Monthly sample:


,month,month_end,cpih_idx,inf_yoy_pct_m,inf_mom_pct_m
0,1988-01,1988-01-31,46.9,NaN,NaN
1,1988-02,1988-02-29,47.0,NaN,0.212993
2,1988-03,1988-03-31,47.2,NaN,0.424629
3,1988-04,1988-04-30,47.8,NaN,1.263175
4,1988-05,1988-05-31,48.0,NaN,0.417537
5,1988-06,1988-06-30,48.2,NaN,0.415801
6,1988-07,1988-07-31,48.2,NaN,0.000000
7,1988-08,1988-08-31,48.5,NaN,0.620478
8,1988-09,1988-09-30,48.7,NaN,0.411523
9,1988-10,1988-10-31,49.0,NaN,0.614127


Quarterly sample:


,quarter,quarter_end,cpih_idx_q,inf_qoq_pct_q,inf_yoy_pct_q,is_partial_q
0,1997Q1,1997-03-31,69.300000,NaN,NaN,0
1,1997Q2,1997-06-30,69.933333,0.909750,NaN,0
2,1997Q3,1997-09-30,70.266667,0.475512,NaN,0
3,1997Q4,1997-12-31,70.633333,0.520465,NaN,0
4,1998Q1,1998-03-31,70.600000,-0.047203,1.858524,0
5,1998Q2,1998-06-30,71.366667,1.080076,2.028850,0
6,1998Q3,1998-09-30,71.433333,0.093371,1.646709,0
7,1998Q4,1998-12-31,71.833333,0.558401,1.684644,0


## 6) series-030725 (2).csv (ONS code EBAQ – UK resident population; “Qtly data interpolated”)

In [9]:
from pathlib import Path
import csv, re, datetime as dt
import pandas as pd
import numpy as np
from IPython.display import display

# ------------------------------------------------------------
# Project paths
# ------------------------------------------------------------
PROJ_ROOT = Path.cwd().resolve()
DATA_RAW = PROJ_ROOT / "data" / "raw"
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJ_ROOT)
print("Raw:", DATA_RAW)
print("Interim:", DATA_INTERIM)

# ------------------------------------------------------------
# 0) CONFIG — source files from data/raw
# ------------------------------------------------------------
# Prefer exact names; fall back to close matches if needed
ebaq_file = DATA_RAW / "series-030725 (2).csv"
if not ebaq_file.exists():
    candidates = sorted(DATA_RAW.glob("series-030725*2*.csv")) or \
                 sorted(DATA_RAW.glob("series-030725*.csv"))
    if not candidates:
        raise FileNotFoundError("Could not find 'series-030725 (2).csv' in data/raw.")
    ebaq_file = candidates[0]
print(f"Using EBAQ source: {ebaq_file}")

# Optional: ONS MYE workbook to splice 2022–2023 anchors (UK total population)
mye_file = DATA_RAW / "ukpopulationestimates183820231.xlsx"
if not mye_file.exists():
    # allow running without MYE; splice will be skipped unless needed
    print("Note: MYE workbook not found in data/raw (optional). Splice will be skipped if not needed.")

ANALYSIS_START_Q = '1997Q1'   # generous buffer
ANALYSIS_END_Q   = '2025Q4'   # safe cap; final merge will align windows

# ------------------------------------------------------------
# 1) HELPERS
# ------------------------------------------------------------
def load_ons_single_series(path: Path) -> pd.DataFrame:
    """
    Load an ONS single-series CSV (annual and/or quarterly lines).
    Returns columns: period, value, year, q (Int64, may be NA).
    """
    text = path.read_text(encoding='utf-8', errors='ignore').splitlines()
    rows = [next(csv.reader([ln])) for ln in text if ln.strip()]
    data = [(r[0].strip('"'), r[1]) for r in rows
            if len(r) >= 2 and re.match(r'^\d{4}(?:\s+Q[1-4])?$', r[0].strip('"'))]
    if not data:
        raise ValueError("No ONS data rows found (expected 'YYYY' or 'YYYY Qn').")
    df = pd.DataFrame(data, columns=['period','value'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['year']  = df['period'].str.extract(r'(\d{4})').astype(int)
    df['q']     = df['period'].str.extract(r'Q([1-4])').astype('Int64')
    return df

def qkey(y:int, q:int) -> str:
    return f"{y}Q{q}"

def q_from_key(qk:str):
    y, q = qk.split('Q'); return int(y), int(q)

def q_end_date(qk:str) -> pd.Timestamp:
    y, q = q_from_key(qk)
    month = {1:3, 2:6, 3:9, 4:12}[q]
    return pd.Timestamp(year=y, month=month, day=1) + pd.offsets.MonthEnd(0)

def qrank(qk:str) -> int:
    y,q = q_from_key(qk); return y*10 + q

def q_range(q_start:str, q_end:str):
    s = qrank(q_start); e = qrank(q_end)
    if e < s: return []
    out = []
    for r in range(s, e+1):
        y, q = divmod(r, 10)
        out.append(qkey(y, q))
    return out

# ------------------------------------------------------------
# 2) LOAD EBAQ AND KEEP QUARTERLY
# ------------------------------------------------------------
ebaq_raw = load_ons_single_series(ebaq_file)

# Keep only quarterly rows (drop annual if present)
ebaq_q = ebaq_raw[ebaq_raw['q'].notna()].copy()
if ebaq_q.empty:
    raise ValueError("This EBAQ file appears to contain only annual values. "
                     "Please download the quarterly-interpolated EBAQ series (YYYY Qn rows).")

# Canonical quarter key and end date
ebaq_q['quarter'] = ebaq_q.apply(lambda r: qkey(int(r['year']), int(r['q'])), axis=1)
ebaq_q['quarter_end'] = ebaq_q['quarter'].apply(q_end_date)

# Units: EBAQ is typically in THOUSANDS of persons
ebaq_q = ebaq_q.rename(columns={'value':'pop_thous'})
ebaq_q['source'] = 'EBAQ'

# Trim window and sort
mask = (ebaq_q['quarter'].apply(qrank) >= qrank(ANALYSIS_START_Q)) & \
       (ebaq_q['quarter'].apply(qrank) <= qrank(ANALYSIS_END_Q))
ebaq_q = ebaq_q.loc[mask].sort_values('quarter').reset_index(drop=True)

# ------------------------------------------------------------
# 3) OPTIONAL SPLICE 2022–2023 USING MYE ANCHORS (Q2 each year)
# ------------------------------------------------------------
def read_mye_uk_totals(mye_xlsx: Path):
    """
    Tries to read UK total population for 2022 and 2023 from the MYE workbook.
    Returns a dict {2022: value_thous, 2023: value_thous} or {} if not found.
    The workbook formats vary; this scans all sheets for a row containing 'United Kingdom'
    and numeric year columns.
    """
    out = {}
    try:
        xls = pd.read_excel(mye_xlsx, sheet_name=None, header=None)
    except Exception as e:
        print(f"MYE workbook not read ({e}). Skipping splice.")
        return out

    for name, sh in xls.items():
        df = sh.copy()
        # Try to locate a row containing 'United Kingdom' (case-insensitive)
        mask_row = df.apply(lambda r: r.astype(str).str.contains('United Kingdom', case=False, na=False).any(), axis=1)
        if not mask_row.any():
            continue
        # Promote the first non-empty row above as header if likely
        header_idx = None
        for i in range(min(10, len(df))):
            vals = df.iloc[i].tolist()
            years_found = sum([1 for v in vals if str(v).strip().isdigit() and 1800 <= int(str(v).strip()) <= 2100])
            if years_found >= 5:
                header_idx = i
                break
        if header_idx is None:
            header_idx = 0
        df2 = pd.read_excel(mye_xlsx, sheet_name=name, header=header_idx)
        # Find UK row
        uk_row = None
        for col in df2.columns:
            if isinstance(col, str) and re.search(r'united\s*kingdom|uk$', col, flags=re.I):
                uk_row = df2[df2[col].astype(str).str.contains('United Kingdom', case=False, na=False)]
                if not uk_row.empty:
                    break
        if uk_row is None or uk_row.empty:
            # try first column as territory name
            uk_row = df2[df2.iloc[:,0].astype(str).str.contains('United Kingdom', case=False, na=False)]
            if uk_row.empty:
                continue

        # Collect year columns 2022 and 2023 if present
        for y in (2022, 2023):
            if y in df2.columns:
                val = pd.to_numeric(uk_row.iloc[0][y], errors='coerce')
                if pd.notna(val):
                    out[y] = float(val) / 1000.0 if val > 1e7 else float(val)  # if persons, convert to thousands
        if all(y in out for y in (2022, 2023)):
            break

    if not out:
        print("Could not locate UK totals for 2022/2023 in MYE workbook; splice skipped.")
    else:
        print(f"MYE UK anchors found (thousands): {out}")
    return out

# Attempt splice if necessary (EBAQ often stops at 2021Q4)
last_q_available = ebaq_q['quarter'].iloc[-1]
anchors = {}
if mye_file.exists() and qrank(last_q_available) < qrank('2022Q2'):
    anchors = read_mye_uk_totals(mye_file)

def splice_with_mye(ebaq_df: pd.DataFrame, anchors_dict: dict) -> pd.DataFrame:
    """
    Splice quarters from last available to 2023Q4 using MYE Q2 anchors.
    Piecewise linear interpolation between:
      last_q -> 2022Q2 -> 2023Q2, then linear within H2 2023 to 2023Q4.
    """
    if not anchors_dict:
        return ebaq_df

    df = ebaq_df.copy()
    last_q = df['quarter'].iloc[-1]
    last_val = df['pop_thous'].iloc[-1]

    # Build anchor points (quarter -> value_thous)
    points = { last_q: last_val }
    if 2022 in anchors_dict:
        points['2022Q2'] = anchors_dict[2022]
    if 2023 in anchors_dict:
        points['2023Q2'] = anchors_dict[2023]

    # Generate required target quarters up to 2023Q4
    target_end = '2023Q4' if '2023Q2' in points else '2022Q4'
    new_quarters = [q for q in q_range(last_q, target_end) if q not in df['quarter'].tolist()]
    if not new_quarters:
        return df

    # Helper: linear interpolate across quarter ranks
    def lin_interp(q1, v1, q2, v2, qk):
        r1, r2, rk = qrank(q1), qrank(q2), qrank(qk)
        if r2 == r1:
            return v1
        frac = (rk - r1) / (r2 - r1)
        return v1 + frac * (v2 - v1)

    # Build segments:
    segments = []
    # last_q -> 2022Q2
    if '2022Q2' in points and qrank('2022Q2') > qrank(last_q):
        segments.append((last_q, points[last_q], '2022Q2', points['2022Q2']))
    # 2022Q2 -> 2023Q2
    if '2023Q2' in points and qrank('2023Q2') > qrank('2022Q2'):
        segments.append(('2022Q2', points['2022Q2'], '2023Q2', points['2023Q2']))
    # 2023Q2 -> 2023Q4 (simple linear within H2 2023)
    if '2023Q2' in points:
        segments.append(('2023Q2', points['2023Q2'], '2023Q4', points['2023Q2'] * (1 + 0.003)))  # small drift (0.3%) over H2

    # Compute values for each missing quarter from segments
    est = {}
    for q1, v1, q2, v2 in segments:
        for qk_ in q_range(q1, q2):
            if qk_ in (q1,) or qk_ in df['quarter'].tolist():
                continue
            est[qk_] = lin_interp(q1, v1, q2, v2, qk_)

    if not est:
        return df

    add = pd.DataFrame({
        'quarter': list(est.keys()),
        'pop_thous': list(est.values()),
        'source': 'EBAQ+MYE splice'
    })
    add['quarter_end'] = add['quarter'].apply(q_end_date)
    df2 = (pd.concat([df, add], ignore_index=True)
             .sort_values('quarter')
             .reset_index(drop=True))
    return df2

if anchors:
    ebaq_q = splice_with_mye(ebaq_q, anchors)

# ------------------------------------------------------------
# 4) DERIVED FIELDS & QA
# ------------------------------------------------------------
# Persons, logs, YoY
ebaq_q['pop_persons'] = ebaq_q['pop_thous'] * 1000.0
ebaq_q['ln_pop']      = np.log(ebaq_q['pop_persons'])
ebaq_q = ebaq_q.sort_values('quarter').reset_index(drop=True)
ebaq_q['pop_yoy_pct'] = 100 * (ebaq_q['ln_pop'] - ebaq_q['ln_pop'].shift(4))

# Integrity: monotonicity (allow tiny dips due to revisions; just flag)
delta = ebaq_q['pop_persons'].diff()
if (delta < -1000).any():  # drop more than ~1k persons QoQ as a red flag
    print("WARNING: Detected quarter-on-quarter population decreases larger than ~1k persons. Inspect around:")
    display(ebaq_q.loc[delta < -1000, ['quarter','pop_persons']])

# ------------------------------------------------------------
# 5) FINALIZE & SAVE (to data/interim)
# ------------------------------------------------------------
# Select and trim columns
pop_out = ebaq_q[['quarter','quarter_end','pop_thous','pop_persons','ln_pop','pop_yoy_pct','source']].copy()

# Trim to configured analysis window
mask2 = (pop_out['quarter'].apply(qrank) >= qrank(ANALYSIS_START_Q)) & \
        (pop_out['quarter'].apply(qrank) <= qrank(ANALYSIS_END_Q))
pop_out = pop_out.loc[mask2].reset_index(drop=True)

# Save
f_pop = DATA_INTERIM / 'pop_q.csv'
pop_out.to_csv(f_pop, index=False)
print(f"Saved population (quarterly) to: {f_pop}")

# Preview
display(pop_out.head(8))
display(pop_out.tail(8))

Project root: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code
Raw: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw
Interim: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim
Using EBAQ source: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\raw\series-030725 (2).csv
Note: MYE workbook not found in data/raw (optional). Splice will be skipped if not needed.
Saved population (quarterly) to: C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\interim\pop_q.csv


,quarter,quarter_end,pop_thous,pop_persons,ln_pop,pop_yoy_pct,source
0,1997Q1,1997-03-31,58277,58277000.0,17.880718,NaN,EBAQ
1,1997Q2,1997-06-30,58314,58314000.0,17.881353,NaN,EBAQ
2,1997Q3,1997-09-30,58354,58354000.0,17.882038,NaN,EBAQ
3,1997Q4,1997-12-31,58395,58395000.0,17.882741,NaN,EBAQ
4,1998Q1,1998-03-31,58435,58435000.0,17.883426,0.270752,EBAQ
5,1998Q2,1998-06-30,58475,58475000.0,17.884110,0.275711,EBAQ
6,1998Q3,1998-09-30,58527,58527000.0,17.884999,0.296028,EBAQ
7,1998Q4,1998-12-31,58580,58580000.0,17.885904,0.316307,EBAQ


,quarter,quarter_end,pop_thous,pop_persons,ln_pop,pop_yoy_pct,source
104,2023Q1,2023-03-31,68270,68270000.0,18.038981,1.209867,EBAQ
105,2023Q2,2023-06-30,68492,68492000.0,18.042228,1.304980,EBAQ
106,2023Q3,2023-09-30,68676,68676000.0,18.044910,1.245419,EBAQ
107,2023Q4,2023-12-31,68859,68859000.0,18.047571,1.184760,EBAQ
108,2024Q1,2024-03-31,69043,69043000.0,18.050240,1.125907,EBAQ
109,2024Q2,2024-06-30,69226,69226000.0,18.052887,1.065956,EBAQ
110,2024Q3,2024-09-30,69387,69387000.0,18.055210,1.029974,EBAQ
111,2024Q4,2024-12-31,69547,69547000.0,18.057513,0.994185,EBAQ


## one standardized master panel

In [10]:
from pathlib import Path
import pandas as pd

PROJ_ROOT = Path.cwd().resolve()
DATA_INTERIM = PROJ_ROOT / "data" / "interim"
DATA_PROCESSED = PROJ_ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

frames = []

# --- Map each interim file -> standard (measure, period, value) on QUARTER keys ---
# 1) Defence totals (quarterly nominal, long)
f = DATA_INTERIM / "defence_q_nominal_long.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","nominal_q_mn":"value","component":"measure"})
    # keep only total/current/capital as separate measures
    frames.append(df[["measure","period","value"]])

# 2) Nominal GDP (YBHA)
f = DATA_INTERIM / "ybha_q.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","gdp_nom_mn":"value"})
    df["measure"] = "gdp_nominal_mn"
    frames.append(df[["measure","period","value"]])

# 3) Real GDP level (ABMI)
f = DATA_INTERIM / "abmi_q.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","gdp_real_mn":"value"})
    df["measure"] = "gdp_real_mn"
    frames.append(df[["measure","period","value"]])

# 4) GDP implied deflator (YBGB) – as-published index (base per ONS)
f = DATA_INTERIM / "ybgb_q.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","deflator_index":"value"})
    df["measure"] = "gdp_deflator_index"
    frames.append(df[["measure","period","value"]])

# (Optional) FY-anchored deflator variant; keep as a separate measure
f = DATA_INTERIM / "ybgb_q_fyanchored.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","deflator_index_fyanchored":"value"})
    df["measure"] = "gdp_deflator_index_fyanchored"
    frames.append(df[["measure","period","value"]])

# 5) CPIH (quarterly average index)
f = DATA_INTERIM / "cpih_quarterly.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","cpih_idx_q":"value"})
    df["measure"] = "cpih_index_qavg"
    frames.append(df[["measure","period","value"]])

# 6) Population (EBAQ)
f = DATA_INTERIM / "pop_q.csv"
if f.exists():
    df = pd.read_csv(f)
    df = df.rename(columns={"quarter":"period","pop_thous":"value"})
    df["measure"] = "population_thousands"
    frames.append(df[["measure","period","value"]])

# Combine & tidy
if frames:
    master = pd.concat(frames, ignore_index=True)
    master["period"] = master["period"].astype(str)
    master = master.dropna(subset=["period","value"]).sort_values(["measure","period"]).reset_index(drop=True)
    out_path = DATA_PROCESSED / "master_panel.csv"
    master.to_csv(out_path, index=False)
    print(f"Wrote master panel -> {out_path} ({len(master):,} rows across {master['measure'].nunique()} measures)")
else:
    print("No interim files found. Make sure the six source scripts have written their outputs to /data/interim.")

Wrote master panel -> C:\Users\Aniruddha\Desktop\Business Project_BEMM466\Project Code\data\processed\master_panel.csv (776 rows across 8 measures)


## Augmented Master Panel (real series via YBGB, per‑capita, and QoQ/YoY growth)

In [11]:
# Robust, error-tolerant augmentation script
# - Builds real MoD series using the GDP implied deflator (YBGB)
# - Derives real GDP if missing (from YBHA & YBGB)
# - Adds per‑capita versions
# - Adds QoQ and YoY growth rates for GDP and defence (total, capex, current)
# - Never hard-fails on missing columns: it computes whatever is feasible and prints a clear summary.
#
# Expected to run in your project root with a master panel already created.
# If `df` is defined in memory, it will use that. Otherwise it will try to load from data/processed/*.
from __future__ import annotations

from pathlib import Path
import pandas as pd
import numpy as np

# ----------------------------- Configuration -----------------------------

DATA_PROCESSED = Path("data/processed")
today = pd.Timestamp.today().date()
OUT_PARQUET = DATA_PROCESSED / f"master_panel_{today:%Y%m%d}.parquet"
OUT_CSV     = DATA_PROCESSED / f"master_panel_{today:%Y%m%d}.csv"

POSSIBLE_MASTERS = [
    DATA_PROCESSED / "master_panel.parquet",
    DATA_PROCESSED / "master_panel_latest.parquet",
    DATA_PROCESSED / "master_panel.csv",
]

# Candidate names (very liberal). Matching is case-insensitive and punctuation-agnostic.
CANDIDATES = {
    "period": [
        "quarter","period","date","obs_date","time","quarter_str","quarterly",
        "qtr","year_quarter","yearq","date_q","period_q","period_quarter"
    ],
    "gdp_real": [
        "gdp_real_abmi","abmi","gdp_cvm_abmi","gdp_cvm","gdp_chain_volume","gdp_volume",
        "gdp_real","real_gdp","gdpr","rgdp","gdp_cvm_level","gdp_cvm_gbp","abmi_cvm"
    ],
    "gdp_nominal": [
        "gdp_nominal_ybha","ybha","gdp_current_prices","gdp_nominal","gdp_cp",
        "ngdp","gdp_value","gdp_curr","gdp_current"
    ],
    "deflator": [
        "gdp_deflator_ybgb","ybgb","gdp_deflator","gdp_deflator_index",
        "gdp_implied_deflator","implied_deflator_ybgb","ybgb_q","gdp_deflator_index_2019_100"
    ],
    "population": [
        "population_q","population","uk_population_q","pop_q","pop","uk_pop",
        "population_thousands","population_millions","resident_population_q","population_estimate_q"
    ],
    "mod_total_nom": [
        "mod_total_nom","mod_total_nominal","mod_total_current_prices","mod_total","defence_total_nom",
        "defence_total_current_prices","defence_total_cp","mod_total_cp","def_exp_total_cp","defence_total"
    ],
    "mod_cap_nom": [
        "mod_cap_nom","mod_capital_nominal","mod_capital","mod_capex_nom","defence_capital_nom",
        "mod_equipment_nom","mod_equipment_cp","defence_capex_cp","equipment_expenditure_cp"
    ],
    "mod_cur_nom": [
        "mod_cur_nom","mod_current_nominal","mod_current","mod_recurrent_nom","defence_current_nom",
        "mod_resource_nom","mod_resource_cp","defence_current_cp","mod_current_cp"
    ],
}

# ------------------------------- Helpers ---------------------------------

def _norm(s: str) -> str:
    """Normalize a column name: lowercase + alnum only."""
    return "".join(ch for ch in s.lower() if ch.isalnum())

def _build_norm_map(df: pd.DataFrame) -> dict[str, str]:
    """Map normalized names -> original df column names (first occurrence wins)."""
    m = {}
    for c in df.columns:
        nc = _norm(c)
        if nc and nc not in m:
            m[nc] = c
    return m

def _find_col(df: pd.DataFrame, names: list[str]) -> str | None:
    """Return the first matching column by normalized name; None if not found."""
    nm = _build_norm_map(df)
    for cand in names:
        nc = _norm(cand)
        if nc in nm:
            return nm[nc]
    return None

def _coerce_numeric(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s.astype(str).str.replace(",", ""), errors="coerce")

def _ensure_period_index(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure a quarterly PeriodIndex named 'quarter'."""
    if isinstance(df.index, pd.PeriodIndex) and df.index.freqstr and df.index.freqstr.upper().startswith("Q"):
        df.index.name = "quarter"
        return df.sort_index()

    # try to find a period-ish column
    pcol = _find_col(df, CANDIDATES["period"])
    if pcol is None:
        # As a last resort, try any column that looks like 'YYYY-Qn' strings or datetimes
        for c in df.columns:
            ser = df[c].astype(str)
            if ser.str.contains(r"^\d{4}Q[1-4]$", regex=True, na=False).any():
                pcol = c
                break
        if pcol is None:
            raise ValueError("No quarterly period column found. Please add one (e.g., 'quarter' like '2016Q3').")

    x = pd.to_datetime(df[pcol], errors="coerce")
    if x.notna().any():
        qidx = x.dt.to_period("Q")
    else:
        # likely 'YYYYQn' strings
        qidx = pd.PeriodIndex(df[pcol].astype(str), freq="Q")
    out = df.set_index(qidx).sort_index()
    out.index.name = "quarter"
    return out

def _deflator_factor(deflator: pd.Series) -> pd.Series:
    """Return multiplicative price factor from an index or factor."""
    d = _coerce_numeric(deflator).copy()
    med = d.median(skipna=True)
    if pd.isna(med):
        raise ValueError("Deflator series has no numeric values.")
    return (d / 100.0) if med > 10 else d

def _detect_population_scale(pop: pd.Series) -> int:
    """If median < 1e6 assume thousands → ×1000, else assume persons."""
    med = pop.median(skipna=True)
    if pd.isna(med):
        return 1
    return 1000 if med < 1_000_000 else 1

def _pct_change(s: pd.Series, periods: int) -> pd.Series:
    out = s.div(s.shift(periods)).sub(1)
    return out.replace([np.inf, -np.inf], np.nan)

def _safe_add(df: pd.DataFrame, col: str, series: pd.Series) -> None:
    """Add/overwrite a column aligning on index."""
    df[col] = series.reindex(df.index)

# ------------------------------- Load ------------------------------------

try:
    df  # noqa: F821
except NameError:
    df = None

if df is None:
    for p in POSSIBLE_MASTERS:
        if p.exists():
            df = pd.read_parquet(p) if p.suffix == ".parquet" else pd.read_csv(p)
            print(f"Loaded master from: {p}")
            break
    if df is None:
        raise FileNotFoundError(
            "Master panel not found. Ensure a DataFrame `df` exists, or place one at data/processed/master_panel.*"
        )

# ------------------------------ Augment ----------------------------------

df = _ensure_period_index(df)
messages = []

# Locate columns (case/format tolerant)
gdp_real_col = _find_col(df, CANDIDATES["gdp_real"])
gdp_nom_col  = _find_col(df, CANDIDATES["gdp_nominal"])
defl_col     = _find_col(df, CANDIDATES["deflator"])
pop_col      = _find_col(df, CANDIDATES["population"])
mod_tot_col  = _find_col(df, CANDIDATES["mod_total_nom"])
mod_cap_col  = _find_col(df, CANDIDATES["mod_cap_nom"])
mod_cur_col  = _find_col(df, CANDIDATES["mod_cur_nom"])

messages.append("Matched columns (None means not found):")
messages.append(f"  • GDP real:      {gdp_real_col}")
messages.append(f"  • GDP nominal:   {gdp_nom_col}")
messages.append(f"  • GDP deflator:  {defl_col}")
messages.append(f"  • Population:    {pop_col}")
messages.append(f"  • MoD total CP:  {mod_tot_col}")
messages.append(f"  • MoD capex CP:  {mod_cap_col}")
messages.append(f"  • MoD current CP:{mod_cur_col}")

# Prepare deflator factor if available
price_factor = None
if defl_col is not None:
    deflator_ser = _coerce_numeric(df[defl_col]).ffill().bfill()
    price_factor = _deflator_factor(deflator_ser)
else:
    messages.append("  ! Deflator missing → cannot deflate nominal series.")

# -------- GDP real (prefer existing; else derive from nominal & deflator) --------
gdp_real_name = None
if gdp_real_col is not None:
    gdp_real = _coerce_numeric(df[gdp_real_col])
    gdp_real_name = gdp_real_col  # keep original name
else:
    if (gdp_nom_col is not None) and (price_factor is not None):
        gdp_nom = _coerce_numeric(df[gdp_nom_col])
        gdp_real = gdp_nom / price_factor
        gdp_real_name = "gdp_real_abmi"  # standardize output name
        _safe_add(df, gdp_real_name, gdp_real)
        messages.append(f"  • Derived real GDP as `{gdp_real_name}` from {gdp_nom_col}/{defl_col}.")
    else:
        gdp_real = None
        messages.append("  ! Could not obtain real GDP (need either an existing real series OR nominal+deflator).")

# -------- Deflate MoD nominal series to real (2019=100 style if YBGB) --------
def add_mod_real(nom_col: str, out_name: str):
    if nom_col is None:
        messages.append(f"  ! Missing nominal series for {out_name} → skipped.")
        return
    if price_factor is None:
        messages.append(f"  ! Missing deflator for {out_name} → skipped.")
        return
    nom = _coerce_numeric(df[nom_col])
    _safe_add(df, out_name, nom / price_factor)
    messages.append(f"  • Created {out_name} from {nom_col}/{defl_col}.")

add_mod_real(mod_tot_col, "mod_total_real")
add_mod_real(mod_cap_col, "mod_cap_real")
add_mod_real(mod_cur_col, "mod_cur_real")

# -------- Per‑capita versions (if population available) --------
if pop_col is not None:
    pop = _coerce_numeric(df[pop_col]).ffill().bfill()
    pop_scale = _detect_population_scale(pop)
    persons = pop * pop_scale

    # GDP per-capita (if we have real GDP)
    if gdp_real_name is not None:
        _safe_add(df, "gdp_real_pc", df[gdp_real_name] / persons)
        messages.append("  • Added gdp_real_pc.")
    else:
        messages.append("  ! Skipped gdp_real_pc (no real GDP).")

    # MoD per-capita (if real series exist)
    for base in ["mod_total_real", "mod_cap_real", "mod_cur_real"]:
        if base in df.columns:
            _safe_add(df, f"{base}_pc", df[base] / persons)
            messages.append(f"  • Added {base}_pc.")
else:
    messages.append("  ! Population missing → skipped all per‑capita calculations.")

# -------- Growth rates (QoQ & YoY) for GDP (real) and MoD real series --------
def add_growth(series_name: str, label: str | None = None):
    if series_name not in df.columns:
        messages.append(f"  ! Growth skipped (missing series): {series_name}")
        return
    base = label or series_name
    s = _coerce_numeric(df[series_name])
    df[f"{base}_qoq_pct"] = _pct_change(s, 1)
    df[f"{base}_yoy_pct"] = _pct_change(s, 4)
    messages.append(f"  • Added QoQ/YoY growth for {base}.")

# GDP growth (use the detected/derived real GDP)
if gdp_real_name is not None:
    add_growth(gdp_real_name)  # columns: <name>_qoq_pct, <name>_yoy_pct
else:
    messages.append("  ! Skipped GDP growth (no real GDP).")

# Defence growth (real)
for base in ["mod_total_real", "mod_cap_real", "mod_cur_real"]:
    if base in df.columns:
        add_growth(base)

# ------------------------------- Save -----------------------------------

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
df.to_parquet(OUT_PARQUET)
df.to_csv(OUT_CSV, index=True)

print("\n".join(messages))
print("\nSaved:")
print(f"  - {OUT_PARQUET}")
print(f"  - {OUT_CSV}")

Matched columns (None means not found):
  • GDP real:      None
  • GDP nominal:   None
  • GDP deflator:  None
  • Population:    None
  • MoD total CP:  None
  • MoD capex CP:  None
  • MoD current CP:None
  ! Deflator missing → cannot deflate nominal series.
  ! Could not obtain real GDP (need either an existing real series OR nominal+deflator).
  ! Missing nominal series for mod_total_real → skipped.
  ! Missing nominal series for mod_cap_real → skipped.
  ! Missing nominal series for mod_cur_real → skipped.
  ! Population missing → skipped all per‑capita calculations.
  ! Skipped GDP growth (no real GDP).

Saved:
  - data\processed\master_panel_20250902.parquet
  - data\processed\master_panel_20250902.csv
